# Swinging Atwood's Machine#

A [Swinging Atwood's Machine](https://en.wikipedia.org/wiki/Swinging_Atwood%27s_machine) (SAM) is an example of a dynamical system that can exhibit chaotic behaviours. Strangely, it is know to actually be integrable in the case that the ratio of the masses is [3](https://aapt.scitation.org/doi/10.1119/1.14710), as is conjectured to be so for the general case that the ratio is $4n^2-1$, $n\in\mathbb{N}$. This notebook produces an animation of the motion of a SAM in the case the ratio is 3, based off of the [matplotlib example](https://matplotlib.org/gallery/animation/double_pendulum_sgskip.html#sphx-glr-gallery-animation-double-pendulum-sgskip-py), illustrating numerically that I (the first integral in the case the ratio is 3) is conserved.

In [184]:
from numpy import sin, cos
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import matplotlib.animation as animation
from matplotlib import rc

In [185]:
g=9.8  # acceleration due to gravity.
m=1.0  # freely rotating mass.
M=3.0  # fixed mass.
L=4.0  # Total length of line.
Mpulley=np.array([-1.0,0]) # location of pulley next to fixed mass.
mpulley=np.array([1.0,0])  # location of pulley next to rotating mass.

d=np.linalg.norm(Mpulley-mpulley) # distance between the two pulleys.
mu=M/m #ratio of the masses, a more convenient parameter. 

In [186]:
#State is specified in terms of r, p_r, theta, and p_theta in phase space, and they take the initial values below.
r=1.0
p_r=0.0
theta=np.pi/2
p_theta=0.0

initial_state=np.array([r,p_r,theta,p_theta])

In [187]:
def dH(state, t): #The derivative of the state representing Hamilton's equations.

    ddt=np.zeros_like(state)
    ddt[0]=state[1]/m/(1+mu)
    ddt[1]=state[3]**2/m/state[0]**3-g*m*(mu-cos(state[2]))
    ddt[2]=state[3]/m/state[0]**2
    ddt[3]=-g*m*state[0]*sin(state[2])

    return ddt

In [188]:
def Hamiltonian(state): #Value of the Hamiltonian.
        """compute the energy of the current state"""
        V=g*state[:,0]*m*(mu-cos(state[:,2]))
        T=0.5*(state[:,1]**2/(1+mu)+(state[:,3]/state[:,0])**2)/m

        return T+V

In [189]:
def FirstIntegral(state): #Value of the function I, a first integral for mu=3, found by Tufillaro. This has been split up into a kinetic and potential term to encourage thought. 
    K=state[:,3]*(state[:,1]/(1+mu)*cos(state[:,2]/2)-state[:,3]*sin(state[:,2]/2)/2/state[:,0])/m**2
    U=g*state[:,0]**2*sin(state[:,2]/2)*cos(state[:,2]/2)**2
    
    return K+U

In [190]:
# create a time array from 0 to Tfinal sampled at 0.033 second steps
dt=1/30.0
Tfinal=60.0
t=np.arange(0.0,Tfinal,dt)

In [191]:
# Integrate of Hamilton's equations, giving the cartesian coordinates of the masses.
state=integrate.odeint(dH,initial_state,t)

xM=Mpulley[0]+0*state[:,0]
yM=Mpulley[1]+state[:,0]+d-L

xm=mpulley[0]+state[:,0]*sin(state[:,2])
ym=mpulley[1]-state[:,0]*cos(state[:,2])

H=Hamiltonian(state)
I=FirstIntegral(state)


In [192]:
#Setting up the plot.
fig=plt.figure()

xmin=np.min(np.vstack([xm,xM]))
xmax=np.max(np.vstack([xm,xM]))
ymin=np.min(np.vstack([ym,yM]))
ymax=np.max(np.vstack([ym,yM]))
xmin=xmin-0.1*np.abs(xmin)
ymin=ymin-0.1*np.abs(ymin)
xmax=xmax+0.1*np.abs(xmax)
ymax=ymax+0.1*np.abs(ymax)

ax=fig.add_subplot(111,autoscale_on=False,xlim=(xmin,xmax),ylim=(ymin,ymax))
ax.set_aspect('equal')
ax.grid()

line,=ax.plot([],[],'-',lw=2)
pointM,=ax.plot([],[],marker='o',markersize=5*mu)
pointm,=ax.plot([],[],marker='o',markersize=5)
trace,=ax.plot([],[],color='black',marker='',lw=0.5)
t_template='t = %.1f'
H_template='H = %.2f'
I_template='I = %.2f'
t_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)
H_text = ax.text(0.02, 0.90, '', transform=ax.transAxes)
I_text = ax.text(0.02, 0.85, '', transform=ax.transAxes)

plt.close()

In [193]:
#Defining the functions required for the animation process.
def init():
    line.set_data([],[])
    pointM.set_data([],[])
    pointm.set_data([],[])
    trace.set_data([],[])
    t_text.set_text('')
    H_text.set_text('')
    I_text.set_text('')
    return line, pointM, pointm, trace, t_text, H_text, I_text


def animate(i):
    x=[xM[i],Mpulley[0],mpulley[0],xm[i]]
    y=[yM[i],Mpulley[1],mpulley[1],ym[i]]

    line.set_data(x,y)
    pointM.set_data(xM[i],yM[i])
    pointm.set_data(xm[i],ym[i])
    trace.set_data(xm[0:i],ym[0:i])
    
    t_text.set_text(t_template % (i*dt))
    H_text.set_text(H_template % H[i])
    I_text.set_text(I_template % I[i])
    
    return line, pointM, pointm, trace, t_text, H_text, I_text

In [194]:
#Animation.
ani=animation.FuncAnimation(fig,animate,frames=np.arange(1, len(t)),
                              interval=1000*dt,blit=True,init_func=init)
#ani.save('SAM.mp4', fps=30, extra_args=['-vcodec', 'libx264'])
rc('animation', html='html5')
ani